In [1]:
import pickle
import operator
import time
dataset = 'diginetica'
ds = 'data/diginetica/train-item-views2.csv'

In [2]:
import pandas as pd 
df = pd.read_csv(ds, sep=';')
df['sessionId'] = df['sessionId'].astype('category').cat.codes
df['itemId'] = df['itemId'].astype('category').cat.codes
df[['sessionId', 'itemId']].nunique()
# df.to_csv('./datasets/diginetica/train-item-views.csv', index=False)

sessionId    310324
itemId       122993
dtype: int64

In [3]:
with open(ds, "r") as f:
    if dataset == 'retailrocket' or dataset == 'diginetica':
        lines = f.readlines()[1:]  # skip the first line
    else:
        lines = f.readlines()
    sess_clicks = {}
    sess_date = {}
    for line in lines:
        if dataset == 'retailrocket':
            data = [int(x) for x in line.split()]
            sess_id = data[3]
            item = data[2]
            timestamp = data[0]
        elif dataset == 'yoochoose':
            data = [x for x in line.split(",")]
            sess_id = int(data[0])
            item = int(data[2])
            timestamp = time.mktime(time.strptime(data[1][:19], '%Y-%m-%dT%H:%M:%S'))
        elif dataset == 'diginetica':
            data = [x for x in line.split(";")]
            sess_id = int(data[0])
            timestamp = time.mktime(time.strptime(data[4][:-1], '%Y-%m-%d'))
            timeframe = int(data[3])
            item = (int(data[2]), timestamp, timeframe)

        if sess_id in sess_clicks:
            sess_clicks[sess_id] += [item]
            sess_date[sess_id] = timestamp  # timestamp
        else:
            sess_clicks[sess_id] = [item]
            sess_date[sess_id] = timestamp  # timestamp

    if dataset == 'diginetica':
        for i in list(sess_clicks):
            sorted_clicks = sorted(sess_clicks[i], key=operator.itemgetter(1))
            # sorted_clicks = sorted(sess_clicks[i], key=lambda x: (x[1], x[2]))  #
            sess_clicks[i] = [c[0] for c in sorted_clicks]
print("length of session clicks: %d" % len(sess_clicks))

length of session clicks: 310324


In [4]:
filter_len = 3
for s in list(sess_clicks):
    if len(sess_clicks[s]) <= filter_len:
        del sess_clicks[s]
        del sess_date[s]
print("after filter out length of %d, length of session clicks: %d" % (filter_len, len(sess_clicks)))

# Count number of times each item appears
item_counts = {}
for s in sess_clicks:
    seq = sess_clicks[s]
    for iid in seq:
        if iid in item_counts:
            item_counts[iid] += 1
        else:
            item_counts[iid] = 1

sorted_counts = sorted(item_counts.items(), key=operator.itemgetter(1))

for s in list(sess_clicks):
    curseq = sess_clicks[s]
    filseq = list(filter(lambda i: item_counts[i] >= 5, curseq))
    if len(filseq) <= filter_len:
        del sess_clicks[s]
        del sess_date[s]
    else:
        sess_clicks[s] = filseq
print("after item<5 , length of session clicks: %d" % len(sess_clicks))

# split out train set and test set based on date
dates = list(sess_date.items())
max_timestamp = dates[0][1]

for _, date in dates:
    max_timestamp = max(max_timestamp, date)  # latest date

if dataset == 'retailrocket' or dataset == 'diginetica':
    split_timestamp = max_timestamp - 7 * 86400
else:
    split_timestamp = max_timestamp - 86400

print("split timestamp: %d" % split_timestamp)
train_session = filter(lambda x: x[1] < split_timestamp, dates)
test_session = filter(lambda x: x[1] > split_timestamp, dates)

# Sort sessions by date
train_session = sorted(train_session, key=operator.itemgetter(1))
test_session = sorted(test_session, key=operator.itemgetter(1))

after filter out length of 3, length of session clicks: 121863
after item<5 , length of session clicks: 104676
split timestamp: 1464114600


In [5]:
item_dict = {}
# convert training sessions to sequences and renumber the items
def get_train():
    tra_sid = []
    tra_seq = []
    tra_timestamp = []
    item_cnt = 1
    for s, t in train_session:
        seq = sess_clicks[s]
        outseq = []
        if len(seq) < filter_len:  # Doesn't occur
            continue
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                item_dict[i] = item_cnt
                outseq += [item_dict[i]]
                item_cnt += 1
        tra_sid += [s]
        tra_timestamp += [t]
        tra_seq += [outseq]
    print('item_num: %d' % (item_cnt - 1))
    return tra_sid, tra_timestamp, tra_seq


# Convert test sessions to sequences, ignoring items that do not appear in training set
def get_test():
    tes_sid = []
    tes_seq = []
    tes_timestamp = []
    for s, t in test_session:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < filter_len:
            continue
        tes_sid += [s]
        tes_timestamp += [t]
        tes_seq += [outseq]
    return tes_sid, tes_timestamp, tes_seq


train_sid, train_timestamp, train_seq = get_train()
test_sid, test_timestamp, test_seq = get_test()
print("training: %d" % len(train_sid))
print("testing: %d" % len(test_sid))


# train_data = (train_sid, train_seq, train_timestamp)
# pickle.dump(train_data, open('./datasets/retailrocket/train_seq_2.txt', 'wb'))


# split sequence
def split_seq(sid, timestamp, seq):
    x = []
    t = []
    y = []
    s_id = []
    for sid, seq, timestamp in zip(sid, seq, timestamp):
        if filter_len == 2:
            temp = len(seq) - 1
        else:
            temp = len(seq)
        # for i in range(1, temp):
            i=1
            y += [seq[-i]]
            x += [seq[:-i]]
            t += [timestamp]
            s_id += [sid]
    return x, t, y, s_id


def split_seq_train(sid, timestamp, seq):
    x = []
    t = []
    s_id = []
    for sid, seq, timestamp in zip(sid, seq, timestamp):
        if len(seq) > filter_len:
            x += [seq]
            t += [timestamp]
            s_id += [sid]
        if filter_len == 2:
            temp = len(seq) - 1
        else:
            temp = len(seq)
        for i in range(1, temp):
            x += [seq[:-i]]
            t += [timestamp]
            s_id += [sid]
    return x, t, s_id


tr_seq, tr_timestamp, tr_predict, tr_sid = split_seq(train_sid, train_timestamp, train_seq)
te_seq, te_timestamp, te_predict, te_sid = split_seq(test_sid, test_timestamp, test_seq)

print("after split,training: %d" % len(tr_seq))
print("after split,testing: %d" % len(te_seq))


item_num: 38053
training: 95425
testing: 8143
after split,training: 95425
after split,testing: 8143


In [6]:
def renumber_lists(list_of_lists):
    flat_list = [item for sublist in list_of_lists for item in sublist]
    unique_numbers = sorted(set(flat_list))
    pd.Series(flat_list).astype('category').cat.codes.to_list()
    number_mapping = {num: i for i, num in enumerate(unique_numbers)}
    new_list_of_lists = [[number_mapping[item] for item in sublist] for sublist in list_of_lists]
    return new_list_of_lists

tr_seq = renumber_lists(tr_seq)
tr_sid = pd.Series(tr_sid).astype('category').cat.codes.to_list()

In [7]:

train_data = (tr_sid, tr_seq, tr_timestamp, tr_predict)
pickle.dump(train_data, open('data/diginetica/train_session.txt', 'wb'))
print("finish")


finish
